In [7]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe FaceMesh and Pose
mp_face_mesh = mp.solutions.face_mesh

# Initialize the FaceMesh module
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize normal eye positions (mata normal)
normal_left_eye = None
normal_right_eye = None

# Function to calculate head angle
def calculate_head_angle(left_eye, right_eye):
    angle = math.atan2(right_eye[1] - left_eye[1], right_eye[0] - left_eye[0])
    angle_degrees = math.degrees(angle)
    return angle_degrees

# Capture video from camera
cap = cv2.VideoCapture(0)  # Use 0 for default camera

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform face landmark detection using MediaPipe
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            if normal_left_eye is None and normal_right_eye is None:
                # Set normal eye positions on first frame
                normal_left_eye = (face_landmarks.landmark[159].x, face_landmarks.landmark[159].y)
                normal_right_eye = (face_landmarks.landmark[386].x, face_landmarks.landmark[386].y)

            # Extract coordinates of left and right eyes
            left_eye = (face_landmarks.landmark[159].x, face_landmarks.landmark[159].y)
            right_eye = (face_landmarks.landmark[386].x, face_landmarks.landmark[386].y)

            # Calculate head angle
            angle_degrees = calculate_head_angle(left_eye, right_eye)

            # Compare current eye positions with normal eye positions
            if left_eye[1] > normal_left_eye[1] or right_eye[1] > normal_right_eye[1]:
                status = "Looking down"
            else:
                status = "Not looking down"

            # Draw status on the frame
            cv2.putText(frame, status, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw landmarks on the frame
            for landmark in face_landmarks.landmark:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    cv2.imshow("Head Angle Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:

cap.release()
cv2.destroyAllWindows()
